In [37]:
%load_ext autoreload
%autoreload 2

#import sys
#sys.path.append('../../')

import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

from src.data.helper_closed_transactions import read_epex_file, filter_lead_time, extract_transactions
from src.data.welfare import clean_transactions, read_weekly_prices_file, read_NTC_file, NTC_preparation, match_transactions

#set maximum columns display pandas
pd.options.display.max_columns = 50

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Trying out the complete pipeline with all the compiled functions

In [3]:
df=read_epex_file(
    "../data/external/EPEX_spot_continous_2019/DE Continuous Orders 2019-10/DE Continuous Orders 20191001.csv",
    fast_load=False)

In [17]:
df.head()

,Order ID,Initial ID,Parent ID,Start Validity Date,End Validity Date,Cancelling Date,Delivery Date,Instrument Type,Delivery Instrument,Is block,Area,Side,Price,Volume,Is Executed,Execution Price,Executed Volume,Executed Price & Volume,lead_time,Delivery Start
0,10523814627,10523814627,NaN,2019-09-30 15:00:02.488000+00:00,2019-09-30 15:00:24.095000+00:00,2019-09-30 15:00:24.097000+00:00,2019-10-01 00:00:00+00:00,Hour,0 days 05:00:00,N,DE-AMP,B,-199.00,5.4,0,NaN,0.0,"(nan, 0.0)",0 days 13:59:57.512000,2019-10-01 05:00:00+00:00
1,10523814628,10523814628,NaN,2019-09-30 15:00:02.488000+00:00,2019-09-30 15:00:24.095000+00:00,2019-09-30 15:00:24.097000+00:00,2019-10-01 00:00:00+00:00,Hour,0 days 05:00:00,N,DE-AMP,B,-199.00,5.2,0,NaN,0.0,"(nan, 0.0)",0 days 13:59:57.512000,2019-10-01 05:00:00+00:00
2,10523814629,10523814629,NaN,2019-09-30 15:00:02.488000+00:00,2019-09-30 15:00:24.095000+00:00,2019-09-30 15:00:24.097000+00:00,2019-10-01 00:00:00+00:00,Hour,0 days 05:00:00,N,DE-AMP,S,199.00,5.5,0,NaN,0.0,"(nan, 0.0)",0 days 13:59:57.512000,2019-10-01 05:00:00+00:00
3,10523814630,10523814630,NaN,2019-09-30 15:00:02.488000+00:00,2019-09-30 15:00:24.095000+00:00,2019-09-30 15:00:24.097000+00:00,2019-10-01 00:00:00+00:00,Hour,0 days 05:00:00,N,DE-AMP,S,199.00,5.3,0,NaN,0.0,"(nan, 0.0)",0 days 13:59:57.512000,2019-10-01 05:00:00+00:00
4,10523814686,10523814686,NaN,2019-09-30 15:00:05.586000+00:00,2019-09-30 15:00:18.558000+00:00,2019-09-30 15:00:18.557000+00:00,2019-10-01 00:00:00+00:00,Hour,0 days 01:00:00,N,DE-AMP,B,8.09,5.0,0,NaN,0.0,"(nan, 0.0)",0 days 09:59:54.414000,2019-10-01 01:00:00+00:00


In [4]:
df_filtered=filter_lead_time(df)

In [38]:
pivoted, pivoted_levels=extract_transactions(df_filtered, new_data_type=False)

In [39]:
pivoted_levels.reset_index(inplace=True)

In [40]:
pivoted_levels=clean_transactions(pivoted_levels)

In [41]:
pivoted_levels.shape

(12067, 22)

In [10]:
wp=read_weekly_prices_file("../data/external/Hydro Generation and Price_CH_2019.csv")

In [11]:
# load NTC
NTC = read_NTC_file("../data/external/NTC_DEandCH_2019.csv")
NTC = NTC_preparation(NTC)

/Users/gianlucamancini/Documents/eth_codes/case_study/case_study_2_alpiq/env/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [29]:
pivoted_levels_updated = match_transactions(pivoted_levels, NTC, wp)

In [32]:
pivoted_levels_updated.shape

(12067, 24)

In [31]:
pivoted_levels_updated[pivoted_levels_updated['possible_match']]

,End Validity Date,Executed Price & Volume,Delivery Start,Executed Volume,Execution Price,Initial ID_B,Instrument Type,Is Executed_B,Is block_B,Order ID_B,Parent ID_B,Price_B,Volume_B,lead_time_B,Initial ID_S,Is Executed_S,Is block_S,Order ID_S,Parent ID_S,Price_S,Volume_S,lead_time_S,possible_match,match_binary_outcome
6634,2019-10-01 12:16:40.376000+00:00,"(161.6, 0.05)",2019-10-01 13:00:00+00:00,0.05,161.6,10524642521,Quarter Hour,1,N,10524642890,1.05246e+10,161.6,0.05,0 days 00:43:20.268000,10524642904,2,N,10524642904,NaN,61.6,1.425,0 days 00:43:19.624000,True,0
11706,2019-10-01 22:28:18.536000+00:00,"(158.36, 0.1)",2019-10-01 23:15:00+00:00,0.1,158.36,10525074019,Quarter Hour,1,N,10525074019,NaN,158.86,0.1,0 days 00:46:41.464000,10525073933,2,N,10525073954,1.05251e+10,158.36,0.25,0 days 00:46:54.866000,True,0
10067,2019-10-01 19:26:03.803000+00:00,"(153.0, 0.15)",2019-10-01 20:00:00+00:00,0.15,153,10524956999,Quarter Hour,1,N,10524961742,1.0525e+10,153,0.15,0 days 00:33:57.181000,10524961757,1,N,10524961757,NaN,153,0.15,0 days 00:33:56.197000,True,1
5131,2019-10-01 09:44:09.557000+00:00,"(151.81, 0.05)",2019-10-01 10:15:00+00:00,0.05,151.81,10524534258,Quarter Hour,1,N,10524534258,NaN,151.81,0.05,0 days 00:30:50.491000,10524534260,1,N,10524534260,NaN,151.81,0.05,0 days 00:30:50.443000,True,0
5077,2019-10-01 09:36:09.739000+00:00,"(150.9, 0.05)",2019-10-01 10:15:00+00:00,0.05,150.9,10524529574,Quarter Hour,1,N,10524529574,NaN,150.9,0.05,0 days 00:38:50.291000,10524529579,1,N,10524529579,NaN,150.9,0.05,0 days 00:38:50.261000,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8917,2019-10-01 17:14:48.410000+00:00,"(45.99, 0.65)",2019-10-01 17:45:00+00:00,0.65,45.99,10524865072,Quarter Hour,1,N,10524865072,NaN,46.7,0.65,0 days 00:30:11.590000,10524853097,2,N,10524864777,1.05249e+10,45.99,1.225,0 days 00:30:41.056000,True,1
3251,2019-10-01 06:50:40.976000+00:00,"(45.98, 0.025)",2019-10-01 07:45:00+00:00,0.025,45.98,10524419104,Quarter Hour,2,N,10524419104,NaN,47,7.25,0 days 00:54:19.024000,10524418409,1,N,10524418409,NaN,45.98,0.025,0 days 00:55:15.450000,True,1
6721,2019-10-01 12:35:01.127000+00:00,"(45.98, 0.15)",2019-10-01 13:30:00+00:00,0.15,45.98,10524655783,Quarter Hour,1,N,10524655783,NaN,45.98,0.15,0 days 00:54:58.873000,10524654871,2,N,10524654871,NaN,45.98,0.2,0 days 00:55:56.873000,True,0
6755,2019-10-01 12:40:44.725000+00:00,"(45.98, 0.05)",2019-10-01 13:30:00+00:00,0.05,45.98,10524660218,Quarter Hour,2,N,10524660218,NaN,51.7,2.975,0 days 00:49:15.275000,10524655447,1,N,10524656589,1.05247e+10,45.98,0.05,0 days 00:53:43.147000,True,0


In [33]:
pivoted_levels_updated[pivoted_levels_updated['match_binary_outcome']==1]

,End Validity Date,Executed Price & Volume,Delivery Start,Executed Volume,Execution Price,Initial ID_B,Instrument Type,Is Executed_B,Is block_B,Order ID_B,Parent ID_B,Price_B,Volume_B,lead_time_B,Initial ID_S,Is Executed_S,Is block_S,Order ID_S,Parent ID_S,Price_S,Volume_S,lead_time_S,possible_match,match_binary_outcome
10067,2019-10-01 19:26:03.803000+00:00,"(153.0, 0.15)",2019-10-01 20:00:00+00:00,0.15,153,10524956999,Quarter Hour,1,N,10524961742,1.0525e+10,153,0.15,0 days 00:33:57.181000,10524961757,1,N,10524961757,NaN,153,0.15,0 days 00:33:56.197000,True,1
8930,2019-10-01 17:16:00.432000+00:00,"(148.0, 0.05)",2019-10-01 18:00:00+00:00,0.05,148,10524866072,Quarter Hour,1,N,10524866072,NaN,148,0.05,0 days 00:43:59.814000,10524866075,1,N,10524866075,NaN,148,0.05,0 days 00:43:59.568000,True,1
3269,2019-10-01 06:51:09.418000+00:00,"(133.8, 0.05)",2019-10-01 07:30:00+00:00,0.05,133.8,10524419426,Quarter Hour,1,N,10524419426,NaN,133.8,0.05,0 days 00:38:50.605000,10524419428,1,N,10524419428,NaN,133.8,0.05,0 days 00:38:50.582000,True,1
10085,2019-10-01 19:27:10.382000+00:00,"(133.01, 0.05)",2019-10-01 20:00:00+00:00,0.05,133.01,10524962361,Quarter Hour,1,N,10524962361,NaN,133.01,0.05,0 days 00:32:49.705000,10524962364,1,N,10524962364,NaN,133.01,0.05,0 days 00:32:49.618000,True,1
3480,2019-10-01 07:17:13.828000+00:00,"(124.3, 0.05)",2019-10-01 08:00:00+00:00,0.05,124.3,10524435739,Quarter Hour,1,N,10524436888,1.05244e+10,124.3,0.05,0 days 00:42:46.193000,10524436891,1,N,10524436891,NaN,124.3,0.05,0 days 00:42:46.172000,True,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3644,2019-10-01 07:39:59.379000+00:00,"(46.0, 0.025)",2019-10-01 08:30:00+00:00,0.025,46,10524453477,Quarter Hour,2,N,10524454115,1.05245e+10,200,0.5,0 days 00:50:00.621000,10524447894,1,N,10524453755,1.05245e+10,46,0.025,0 days 00:50:42.459000,True,1
3645,2019-10-01 07:39:59.379000+00:00,"(46.0, 0.025)",2019-10-01 08:30:00+00:00,0.025,46,10524453477,Quarter Hour,2,N,10524454115,1.05245e+10,200,0.475,0 days 00:50:00.621000,10524453721,1,N,10524453721,NaN,46,0.025,0 days 00:50:46.754000,True,1
3646,2019-10-01 07:39:59.379000+00:00,"(46.0, 0.025)",2019-10-01 08:30:00+00:00,0.025,46,10524453477,Quarter Hour,2,N,10524454115,1.05245e+10,200,0.475,0 days 00:50:00.621000,10524447894,1,N,10524453755,1.05245e+10,46,0.025,0 days 00:50:42.459000,True,1
8917,2019-10-01 17:14:48.410000+00:00,"(45.99, 0.65)",2019-10-01 17:45:00+00:00,0.65,45.99,10524865072,Quarter Hour,1,N,10524865072,NaN,46.7,0.65,0 days 00:30:11.590000,10524853097,2,N,10524864777,1.05249e+10,45.99,1.225,0 days 00:30:41.056000,True,1


Income

In [35]:
condition= pivoted_levels_updated['match_binary_outcome']==1
s=pivoted_levels_updated[condition]['Executed Volume']* pivoted_levels_updated[condition]['Execution Price']


s.sum()

61099.464249999925

In [36]:
s=pivoted_levels_updated['Executed Volume']* pivoted_levels_updated['Execution Price']
s.sum()

315718.01700000104